In [1]:
## This file implements neural networks and logistic regression on pyopresabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination 
## of over- and under-sampling method, in case of imbalanced data.
## For fully-connected neural networks, the accuracy is 97.71% for combination data and 100% for over-sampling data
## after regularization for combination data.
## For logistic regression, the accuracy is 97.71% for combination data and 100% for over-sampling data.
## For random forest, the accuracy is 100% for both combination and over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99% for combination data, and 99.09% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/pyopresabs_quant.csv')
df.shape

(255, 539)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.158375
1      0.157875
2      0.310875
3      0.160125
4      0.160500
5      0.205000
6      0.158250
7      0.158375
8      0.166000
9      0.157750
10     0.159375
11     0.159000
12     0.161000
13     0.161875
14     0.157875
15     0.157125
16     0.305000
17     0.281250
18     0.244625
19     0.156875
20     0.169625
21     0.282500
22     0.156750
23     0.161875
24     0.525750
25     0.155375
26     0.171375
27     0.156500
28     0.158625
29     0.288500
         ...   
225    0.179000
226    0.333889
227    0.400222
228    0.499444
229    0.523000
230    0.267250
231    0.209000
232    0.243625
233    0.201875
234    0.160000
235    0.184250
236    0.178000
237    0.157000
238    0.158250
239    0.201000
240    0.232750
241    0.174250
242    0.159250
243    0.160125
244    0.154000
245    0.156875
246    0.233375
247    0.157375
248    0.158375
249    0.158625
250    0.231250
251    0.174625
252    0.157000
253    0.167625
254    0.163000
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,...,group_8666,group_8667,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno
0,107,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.158375
1,109,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.157875
2,115,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.310875
3,120335,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.160125
4,120337,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.160500


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

0    237
1     18
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 539)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 538)

In [13]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,...,group_8666,group_8667,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno
0,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 537) (255,)


In [15]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 201), (1, 234)]


In [16]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 237), (1, 237)]


In [17]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [18]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 304 samples, validate on 131 samples
Epoch 1/1000
304/304 [==============================] - 0s 634us/step - loss: 0.6732 - accuracy: 0.5691 - val_loss: 0.6019 - val_accuracy: 0.6641
Epoch 2/1000
304/304 [==============================] - 0s 130us/step - loss: 0.5874 - accuracy: 0.6875 - val_loss: 0.5520 - val_accuracy: 0.7863
Epoch 3/1000
304/304 [==============================] - 0s 139us/step - loss: 0.5431 - accuracy: 0.7500 - val_loss: 0.5141 - val_accuracy: 0.8015
Epoch 4/1000
304/304 [==============================] - 0s 136us/step - loss: 0.5091 - accuracy: 0.7928 - val_loss: 0.4840 - val_accuracy: 0.8168
Epoch 5/1000
304/304 [==============================] - 0s 169us/step - loss: 0.4810 - accuracy: 0.8059 - val_loss: 0.4591 - val_accuracy: 0.8244
Epoch 6/1000
304/304 [==============================] - 0s 129us/step - loss: 0.4566 - accuracy: 0.8158 - val_loss: 0.4375 - val_accuracy: 0.8550
Epoch 7/1000
304/304 [==============================] - 0s 167us/step - loss: 

304/304 [==============================] - 0s 163us/step - loss: 0.0912 - accuracy: 0.9934 - val_loss: 0.1323 - val_accuracy: 0.9618
Epoch 57/1000
304/304 [==============================] - 0s 109us/step - loss: 0.0891 - accuracy: 0.9967 - val_loss: 0.1311 - val_accuracy: 0.9618
Epoch 58/1000
304/304 [==============================] - 0s 81us/step - loss: 0.0872 - accuracy: 0.9934 - val_loss: 0.1294 - val_accuracy: 0.9695
Epoch 59/1000
304/304 [==============================] - 0s 102us/step - loss: 0.0853 - accuracy: 0.9967 - val_loss: 0.1285 - val_accuracy: 0.9618
Epoch 60/1000
304/304 [==============================] - 0s 102us/step - loss: 0.0838 - accuracy: 0.9934 - val_loss: 0.1267 - val_accuracy: 0.9695
Epoch 61/1000
304/304 [==============================] - 0s 114us/step - loss: 0.0819 - accuracy: 0.9967 - val_loss: 0.1255 - val_accuracy: 0.9695
Epoch 62/1000
304/304 [==============================] - 0s 111us/step - loss: 0.0803 - accuracy: 0.9967 - val_loss: 0.1247 - val_acc

Epoch 112/1000
304/304 [==============================] - 0s 84us/step - loss: 0.0366 - accuracy: 1.0000 - val_loss: 0.0895 - val_accuracy: 0.9847
Epoch 113/1000
304/304 [==============================] - 0s 89us/step - loss: 0.0361 - accuracy: 1.0000 - val_loss: 0.0895 - val_accuracy: 0.9847
Epoch 114/1000
304/304 [==============================] - 0s 90us/step - loss: 0.0357 - accuracy: 1.0000 - val_loss: 0.0892 - val_accuracy: 0.9847
Epoch 115/1000
304/304 [==============================] - 0s 89us/step - loss: 0.0353 - accuracy: 1.0000 - val_loss: 0.0890 - val_accuracy: 0.9847
Epoch 116/1000
304/304 [==============================] - 0s 77us/step - loss: 0.0349 - accuracy: 1.0000 - val_loss: 0.0887 - val_accuracy: 0.9847
Epoch 117/1000
304/304 [==============================] - 0s 85us/step - loss: 0.0344 - accuracy: 1.0000 - val_loss: 0.0883 - val_accuracy: 0.9847
Epoch 118/1000
304/304 [==============================] - 0s 77us/step - loss: 0.0341 - accuracy: 1.0000 - val_loss: 0

304/304 [==============================] - 0s 76us/step - loss: 0.0210 - accuracy: 1.0000 - val_loss: 0.0767 - val_accuracy: 0.9847
Epoch 168/1000
304/304 [==============================] - 0s 75us/step - loss: 0.0209 - accuracy: 1.0000 - val_loss: 0.0765 - val_accuracy: 0.9847
Epoch 169/1000
304/304 [==============================] - 0s 81us/step - loss: 0.0207 - accuracy: 1.0000 - val_loss: 0.0763 - val_accuracy: 0.9847
Epoch 170/1000
304/304 [==============================] - 0s 94us/step - loss: 0.0205 - accuracy: 1.0000 - val_loss: 0.0762 - val_accuracy: 0.9847
Epoch 171/1000
304/304 [==============================] - 0s 138us/step - loss: 0.0203 - accuracy: 1.0000 - val_loss: 0.0760 - val_accuracy: 0.9847
Epoch 172/1000
304/304 [==============================] - 0s 124us/step - loss: 0.0202 - accuracy: 1.0000 - val_loss: 0.0758 - val_accuracy: 0.9847
Epoch 173/1000
304/304 [==============================] - 0s 140us/step - loss: 0.0200 - accuracy: 1.0000 - val_loss: 0.0757 - val_

304/304 [==============================] - 0s 83us/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 0.0706 - val_accuracy: 0.9847
Epoch 223/1000
304/304 [==============================] - 0s 74us/step - loss: 0.0141 - accuracy: 1.0000 - val_loss: 0.0706 - val_accuracy: 0.9847
Epoch 224/1000
304/304 [==============================] - 0s 96us/step - loss: 0.0140 - accuracy: 1.0000 - val_loss: 0.0706 - val_accuracy: 0.9847
Epoch 225/1000
304/304 [==============================] - 0s 80us/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 0.0705 - val_accuracy: 0.9847
Epoch 226/1000
304/304 [==============================] - 0s 79us/step - loss: 0.0138 - accuracy: 1.0000 - val_loss: 0.0704 - val_accuracy: 0.9847
Epoch 227/1000
304/304 [==============================] - 0s 80us/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.0702 - val_accuracy: 0.9847
Epoch 228/1000
304/304 [==============================] - 0s 77us/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.0703 - val_acc

...

304/304 [==============================] - 0s 138us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 776/1000
304/304 [==============================] - 0s 118us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 777/1000
304/304 [==============================] - 0s 105us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 778/1000
304/304 [==============================] - 0s 119us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 779/1000
304/304 [==============================] - 0s 141us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 780/1000
304/304 [==============================] - 0s 114us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 781/1000
304/304 [==============================] - 0s 91us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - v

304/304 [==============================] - 0s 129us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 831/1000
304/304 [==============================] - ETA: 0s - loss: 0.0018 - accuracy: 1.00 - 0s 130us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 832/1000
304/304 [==============================] - 0s 104us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 833/1000
304/304 [==============================] - 0s 89us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 834/1000
304/304 [==============================] - 0s 94us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 835/1000
304/304 [==============================] - 0s 85us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 836/1000
304/304 [==============================] - 0s 75us/step - loss: 0.0024 - 

304/304 [==============================] - 0s 78us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 886/1000
304/304 [==============================] - 0s 74us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 887/1000
304/304 [==============================] - 0s 81us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 888/1000
304/304 [==============================] - 0s 86us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 889/1000
304/304 [==============================] - 0s 78us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 890/1000
304/304 [==============================] - 0s 90us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 891/1000
304/304 [==============================] - 0s 137us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0617 - val_ac

304/304 [==============================] - 0s 113us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 941/1000
304/304 [==============================] - 0s 121us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 942/1000
304/304 [==============================] - 0s 119us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 943/1000
304/304 [==============================] - 0s 113us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 944/1000
304/304 [==============================] - 0s 119us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 945/1000
304/304 [==============================] - 0s 107us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 946/1000
304/304 [==============================] - 0s 170us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0616 - 

304/304 [==============================] - 0s 96us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 996/1000
304/304 [==============================] - 0s 108us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 997/1000
304/304 [==============================] - 0s 92us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 998/1000
304/304 [==============================] - 0s 164us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 999/1000
304/304 [==============================] - 0s 116us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 1000/1000
304/304 [==============================] - 0s 106us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847


In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

131/131 [==============================] - 0s 125us/step
combination test accuracy: 98.47%


In [25]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [26]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 304 samples, validate on 131 samples
Epoch 1/1000
304/304 [==============================] - 0s 850us/step - loss: 2.6363 - accuracy: 0.6349 - val_loss: 0.7668 - val_accuracy: 0.6489
Epoch 2/1000
304/304 [==============================] - 0s 124us/step - loss: 2.4693 - accuracy: 0.7270 - val_loss: 0.7198 - val_accuracy: 0.7023
Epoch 3/1000
304/304 [==============================] - 0s 108us/step - loss: 2.1687 - accuracy: 0.7599 - val_loss: 0.6835 - val_accuracy: 0.7405
Epoch 4/1000
304/304 [==============================] - 0s 132us/step - loss: 2.4337 - accuracy: 0.7566 - val_loss: 0.6560 - val_accuracy: 0.7634
Epoch 5/1000
304/304 [==============================] - 0s 152us/step - loss: 1.6052 - accuracy: 0.8289 - val_loss: 0.6304 - val_accuracy: 0.8550
Epoch 6/1000
304/304 [==============================] - 0s 142us/step - loss: 1.5949 - accuracy: 0.8388 - val_loss: 0.6111 - val_accuracy: 0.8550
Epoch 7/1000
304/304 [==============================] - 0s 102us/step - loss: 

304/304 [==============================] - 0s 121us/step - loss: 2.1223 - accuracy: 0.8717 - val_loss: 0.3404 - val_accuracy: 0.9695
Epoch 57/1000
304/304 [==============================] - 0s 95us/step - loss: 1.8709 - accuracy: 0.8914 - val_loss: 0.3383 - val_accuracy: 0.9695
Epoch 58/1000
304/304 [==============================] - 0s 88us/step - loss: 1.6129 - accuracy: 0.9079 - val_loss: 0.3361 - val_accuracy: 0.9695
Epoch 59/1000
304/304 [==============================] - 0s 80us/step - loss: 1.8642 - accuracy: 0.8914 - val_loss: 0.3337 - val_accuracy: 0.9847
Epoch 60/1000
304/304 [==============================] - 0s 112us/step - loss: 1.3057 - accuracy: 0.9276 - val_loss: 0.3315 - val_accuracy: 0.9847
Epoch 61/1000
304/304 [==============================] - 0s 144us/step - loss: 1.7627 - accuracy: 0.8980 - val_loss: 0.3294 - val_accuracy: 0.9847
Epoch 62/1000
304/304 [==============================] - 0s 155us/step - loss: 2.0115 - accuracy: 0.8816 - val_loss: 0.3283 - val_accur

Epoch 112/1000
304/304 [==============================] - 0s 178us/step - loss: 1.8612 - accuracy: 0.8882 - val_loss: 0.2767 - val_accuracy: 0.9924
Epoch 113/1000
304/304 [==============================] - 0s 124us/step - loss: 1.5562 - accuracy: 0.9079 - val_loss: 0.2757 - val_accuracy: 0.9924
Epoch 114/1000
304/304 [==============================] - 0s 88us/step - loss: 1.8573 - accuracy: 0.8882 - val_loss: 0.2758 - val_accuracy: 0.9924
Epoch 115/1000
304/304 [==============================] - 0s 77us/step - loss: 1.7542 - accuracy: 0.8947 - val_loss: 0.2754 - val_accuracy: 0.9924
Epoch 116/1000
304/304 [==============================] - 0s 73us/step - loss: 2.3156 - accuracy: 0.8586 - val_loss: 0.2743 - val_accuracy: 0.9924
Epoch 117/1000
304/304 [==============================] - 0s 68us/step - loss: 1.5520 - accuracy: 0.9079 - val_loss: 0.2742 - val_accuracy: 0.9924
Epoch 118/1000
304/304 [==============================] - 0s 83us/step - loss: 1.9080 - accuracy: 0.8849 - val_loss:

304/304 [==============================] - 0s 129us/step - loss: 1.7858 - accuracy: 0.8914 - val_loss: 0.2541 - val_accuracy: 0.9924
Epoch 168/1000
304/304 [==============================] - 0s 114us/step - loss: 2.3447 - accuracy: 0.8553 - val_loss: 0.2540 - val_accuracy: 0.9924
Epoch 169/1000
304/304 [==============================] - 0s 110us/step - loss: 1.9380 - accuracy: 0.8816 - val_loss: 0.2537 - val_accuracy: 0.9924
Epoch 170/1000
304/304 [==============================] - 0s 112us/step - loss: 1.6349 - accuracy: 0.9013 - val_loss: 0.2531 - val_accuracy: 0.9924
Epoch 171/1000
304/304 [==============================] - 0s 120us/step - loss: 1.6340 - accuracy: 0.9013 - val_loss: 0.2527 - val_accuracy: 0.9924
Epoch 172/1000
304/304 [==============================] - 0s 98us/step - loss: 2.1919 - accuracy: 0.8651 - val_loss: 0.2527 - val_accuracy: 0.9924
Epoch 173/1000
304/304 [==============================] - 0s 93us/step - loss: 2.1901 - accuracy: 0.8651 - val_loss: 0.2524 - va

Epoch 223/1000
304/304 [==============================] - 0s 71us/step - loss: 1.5670 - accuracy: 0.9046 - val_loss: 0.2412 - val_accuracy: 0.9924
Epoch 224/1000
304/304 [==============================] - 0s 80us/step - loss: 2.0237 - accuracy: 0.8750 - val_loss: 0.2410 - val_accuracy: 0.9924
Epoch 225/1000
304/304 [==============================] - 0s 77us/step - loss: 1.5174 - accuracy: 0.9079 - val_loss: 0.2405 - val_accuracy: 0.9924
Epoch 226/1000
304/304 [==============================] - 0s 80us/step - loss: 1.7672 - accuracy: 0.8914 - val_loss: 0.2407 - val_accuracy: 0.9924
Epoch 227/1000
304/304 [==============================] - 0s 82us/step - loss: 1.3620 - accuracy: 0.9178 - val_loss: 0.2404 - val_accuracy: 0.9924
Epoch 228/1000
304/304 [==============================] - 0s 72us/step - loss: 2.1269 - accuracy: 0.8684 - val_loss: 0.2403 - val_accuracy: 0.9924
Epoch 229/1000
304/304 [==============================] - 0s 79us/step - loss: 2.1244 - accuracy: 0.8684 - val_loss: 0

...

Epoch 726/1000
304/304 [==============================] - 0s 78us/step - loss: 1.5278 - accuracy: 0.9046 - val_loss: 0.2063 - val_accuracy: 0.9771
Epoch 727/1000
304/304 [==============================] - 0s 75us/step - loss: 1.4771 - accuracy: 0.9079 - val_loss: 0.2059 - val_accuracy: 0.9771
Epoch 728/1000
304/304 [==============================] - 0s 73us/step - loss: 1.0712 - accuracy: 0.9342 - val_loss: 0.2056 - val_accuracy: 0.9771
Epoch 729/1000
304/304 [==============================] - 0s 80us/step - loss: 1.5279 - accuracy: 0.9046 - val_loss: 0.2057 - val_accuracy: 0.9771
Epoch 730/1000
304/304 [==============================] - 0s 83us/step - loss: 1.9339 - accuracy: 0.8783 - val_loss: 0.2057 - val_accuracy: 0.9771
Epoch 731/1000
304/304 [==============================] - 0s 76us/step - loss: 2.0859 - accuracy: 0.8684 - val_loss: 0.2056 - val_accuracy: 0.9771
Epoch 732/1000
304/304 [==============================] - 0s 76us/step - loss: 1.7819 - accuracy: 0.8882 - val_loss: 0

Epoch 782/1000
304/304 [==============================] - 0s 102us/step - loss: 1.4247 - accuracy: 0.9112 - val_loss: 0.2045 - val_accuracy: 0.9771
Epoch 783/1000
304/304 [==============================] - 0s 107us/step - loss: 1.5770 - accuracy: 0.9013 - val_loss: 0.2046 - val_accuracy: 0.9771
Epoch 784/1000
304/304 [==============================] - 0s 112us/step - loss: 1.1706 - accuracy: 0.9276 - val_loss: 0.2051 - val_accuracy: 0.9771
Epoch 785/1000
304/304 [==============================] - 0s 118us/step - loss: 1.5256 - accuracy: 0.9046 - val_loss: 0.2040 - val_accuracy: 0.9771
Epoch 786/1000
304/304 [==============================] - 0s 92us/step - loss: 1.4247 - accuracy: 0.9112 - val_loss: 0.2046 - val_accuracy: 0.9771
Epoch 787/1000
304/304 [==============================] - 0s 87us/step - loss: 1.7795 - accuracy: 0.8882 - val_loss: 0.2041 - val_accuracy: 0.9771
Epoch 788/1000
304/304 [==============================] - 0s 81us/step - loss: 1.7296 - accuracy: 0.8914 - val_los

Epoch 838/1000
304/304 [==============================] - 0s 76us/step - loss: 1.8287 - accuracy: 0.8849 - val_loss: 0.2032 - val_accuracy: 0.9771
Epoch 839/1000
304/304 [==============================] - 0s 80us/step - loss: 2.1336 - accuracy: 0.8651 - val_loss: 0.2036 - val_accuracy: 0.9771
Epoch 840/1000
304/304 [==============================] - 0s 79us/step - loss: 1.9308 - accuracy: 0.8783 - val_loss: 0.2034 - val_accuracy: 0.9771
Epoch 841/1000
304/304 [==============================] - 0s 82us/step - loss: 1.1690 - accuracy: 0.9276 - val_loss: 0.2033 - val_accuracy: 0.9771
Epoch 842/1000
304/304 [==============================] - 0s 80us/step - loss: 1.9299 - accuracy: 0.8783 - val_loss: 0.2032 - val_accuracy: 0.9771
Epoch 843/1000
304/304 [==============================] - 0s 75us/step - loss: 1.7269 - accuracy: 0.8914 - val_loss: 0.2030 - val_accuracy: 0.9771
Epoch 844/1000
304/304 [==============================] - 0s 72us/step - loss: 1.7779 - accuracy: 0.8882 - val_loss: 0

Epoch 894/1000
304/304 [==============================] - 0s 80us/step - loss: 1.6752 - accuracy: 0.8947 - val_loss: 0.2024 - val_accuracy: 0.9771
Epoch 895/1000
304/304 [==============================] - 0s 78us/step - loss: 1.2699 - accuracy: 0.9211 - val_loss: 0.2022 - val_accuracy: 0.9771
Epoch 896/1000
304/304 [==============================] - 0s 75us/step - loss: 2.0299 - accuracy: 0.8717 - val_loss: 0.2025 - val_accuracy: 0.9771
Epoch 897/1000
304/304 [==============================] - 0s 79us/step - loss: 1.8273 - accuracy: 0.8849 - val_loss: 0.2026 - val_accuracy: 0.9771
Epoch 898/1000
304/304 [==============================] - 0s 79us/step - loss: 1.9296 - accuracy: 0.8783 - val_loss: 0.2019 - val_accuracy: 0.9771
Epoch 899/1000
304/304 [==============================] - 0s 80us/step - loss: 1.4722 - accuracy: 0.9079 - val_loss: 0.2021 - val_accuracy: 0.9771
Epoch 900/1000
304/304 [==============================] - 0s 77us/step - loss: 1.7767 - accuracy: 0.8882 - val_loss: 0

Epoch 950/1000
304/304 [==============================] - 0s 76us/step - loss: 2.5867 - accuracy: 0.8355 - val_loss: 0.2009 - val_accuracy: 0.9771
Epoch 951/1000
304/304 [==============================] - 0s 72us/step - loss: 2.2321 - accuracy: 0.8586 - val_loss: 0.2006 - val_accuracy: 0.9771
Epoch 952/1000
304/304 [==============================] - 0s 82us/step - loss: 2.1309 - accuracy: 0.8651 - val_loss: 0.2009 - val_accuracy: 0.9771
Epoch 953/1000
304/304 [==============================] - 0s 72us/step - loss: 2.1302 - accuracy: 0.8651 - val_loss: 0.2004 - val_accuracy: 0.9771
Epoch 954/1000
304/304 [==============================] - 0s 84us/step - loss: 2.1292 - accuracy: 0.8651 - val_loss: 0.1998 - val_accuracy: 0.9771
Epoch 955/1000
304/304 [==============================] - 0s 83us/step - loss: 1.7245 - accuracy: 0.8914 - val_loss: 0.2002 - val_accuracy: 0.9771
Epoch 956/1000
304/304 [==============================] - 0s 78us/step - loss: 1.9777 - accuracy: 0.8750 - val_loss: 0

In [28]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

131/131 [==============================] - 0s 51us/step
combination test accuracy: 97.71%


In [29]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [30]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 318us/step - loss: 0.6424 - accuracy: 0.6375 - val_loss: 0.6114 - val_accuracy: 0.6993
Epoch 2/1000
331/331 [==============================] - 0s 71us/step - loss: 0.5881 - accuracy: 0.7100 - val_loss: 0.5736 - val_accuracy: 0.7133
Epoch 3/1000
331/331 [==============================] - 0s 73us/step - loss: 0.5610 - accuracy: 0.7674 - val_loss: 0.5470 - val_accuracy: 0.8182
Epoch 4/1000
331/331 [==============================] - 0s 77us/step - loss: 0.5373 - accuracy: 0.7946 - val_loss: 0.5282 - val_accuracy: 0.7762
Epoch 5/1000
331/331 [==============================] - 0s 74us/step - loss: 0.5179 - accuracy: 0.7946 - val_loss: 0.5131 - val_accuracy: 0.7832
Epoch 6/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4996 - accuracy: 0.8187 - val_loss: 0.5016 - val_accuracy: 0.8042
Epoch 7/1000
331/331 [==============================] - 0s 75us/step - loss: 0.4845

Epoch 57/1000
331/331 [==============================] - 0s 71us/step - loss: 0.1480 - accuracy: 0.9789 - val_loss: 0.1882 - val_accuracy: 0.9720
Epoch 58/1000
331/331 [==============================] - 0s 62us/step - loss: 0.1452 - accuracy: 0.9789 - val_loss: 0.1879 - val_accuracy: 0.9720
Epoch 59/1000
331/331 [==============================] - 0s 69us/step - loss: 0.1423 - accuracy: 0.9819 - val_loss: 0.1838 - val_accuracy: 0.9720
Epoch 60/1000
331/331 [==============================] - 0s 66us/step - loss: 0.1393 - accuracy: 0.9849 - val_loss: 0.1815 - val_accuracy: 0.9720
Epoch 61/1000
331/331 [==============================] - 0s 62us/step - loss: 0.1371 - accuracy: 0.9849 - val_loss: 0.1785 - val_accuracy: 0.9720
Epoch 62/1000
331/331 [==============================] - 0s 64us/step - loss: 0.1344 - accuracy: 0.9849 - val_loss: 0.1751 - val_accuracy: 0.9720
Epoch 63/1000
331/331 [==============================] - 0s 64us/step - loss: 0.1323 - accuracy: 0.9849 - val_loss: 0.1730 -

331/331 [==============================] - 0s 66us/step - loss: 0.0648 - accuracy: 0.9940 - val_loss: 0.0932 - val_accuracy: 0.9860
Epoch 114/1000
331/331 [==============================] - 0s 68us/step - loss: 0.0642 - accuracy: 0.9940 - val_loss: 0.0919 - val_accuracy: 0.9860
Epoch 115/1000
331/331 [==============================] - 0s 69us/step - loss: 0.0636 - accuracy: 0.9940 - val_loss: 0.0911 - val_accuracy: 0.9860
Epoch 116/1000
331/331 [==============================] - 0s 72us/step - loss: 0.0628 - accuracy: 0.9940 - val_loss: 0.0904 - val_accuracy: 0.9860
Epoch 117/1000
331/331 [==============================] - 0s 71us/step - loss: 0.0621 - accuracy: 0.9940 - val_loss: 0.0895 - val_accuracy: 0.9860
Epoch 118/1000
331/331 [==============================] - 0s 80us/step - loss: 0.0615 - accuracy: 0.9940 - val_loss: 0.0886 - val_accuracy: 0.9860
Epoch 119/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0607 - accuracy: 0.9940 - val_loss: 0.0879 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 71us/step - loss: 0.0379 - accuracy: 0.9970 - val_loss: 0.0578 - val_accuracy: 0.9930
Epoch 170/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0375 - accuracy: 0.9970 - val_loss: 0.0577 - val_accuracy: 0.9930
Epoch 171/1000
331/331 [==============================] - 0s 68us/step - loss: 0.0372 - accuracy: 0.9970 - val_loss: 0.0572 - val_accuracy: 0.9930
Epoch 172/1000
331/331 [==============================] - 0s 75us/step - loss: 0.0369 - accuracy: 0.9970 - val_loss: 0.0566 - val_accuracy: 0.9930
Epoch 173/1000
331/331 [==============================] - 0s 76us/step - loss: 0.0367 - accuracy: 0.9970 - val_loss: 0.0565 - val_accuracy: 0.9930
Epoch 174/1000
331/331 [==============================] - 0s 72us/step - loss: 0.0363 - accuracy: 0.9970 - val_loss: 0.0559 - val_accuracy: 0.9930
Epoch 175/1000
331/331 [==============================] - 0s 75us/step - loss: 0.0361 - accuracy: 0.9970 - val_loss: 0

Epoch 225/1000
331/331 [==============================] - 0s 70us/step - loss: 0.0252 - accuracy: 0.9970 - val_loss: 0.0410 - val_accuracy: 1.0000
Epoch 226/1000
331/331 [==============================] - 0s 70us/step - loss: 0.0250 - accuracy: 0.9970 - val_loss: 0.0409 - val_accuracy: 1.0000
Epoch 227/1000
331/331 [==============================] - 0s 69us/step - loss: 0.0249 - accuracy: 0.9970 - val_loss: 0.0407 - val_accuracy: 1.0000
Epoch 228/1000
331/331 [==============================] - 0s 72us/step - loss: 0.0248 - accuracy: 0.9970 - val_loss: 0.0406 - val_accuracy: 1.0000
Epoch 229/1000
331/331 [==============================] - 0s 77us/step - loss: 0.0246 - accuracy: 0.9970 - val_loss: 0.0403 - val_accuracy: 1.0000
Epoch 230/1000
331/331 [==============================] - 0s 65us/step - loss: 0.0244 - accuracy: 0.9970 - val_loss: 0.0399 - val_accuracy: 1.0000
Epoch 231/1000
331/331 [==============================] - 0s 77us/step - loss: 0.0243 - accuracy: 0.9970 - val_loss: 0

...

Epoch 727/1000
331/331 [==============================] - 0s 77us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0108 - val_accuracy: 1.0000
Epoch 728/1000
331/331 [==============================] - 0s 73us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0108 - val_accuracy: 1.0000
Epoch 729/1000
331/331 [==============================] - 0s 71us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0108 - val_accuracy: 1.0000
Epoch 730/1000
331/331 [==============================] - 0s 71us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0107 - val_accuracy: 1.0000
Epoch 731/1000
331/331 [==============================] - 0s 80us/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0107 - val_accuracy: 1.0000
Epoch 732/1000
331/331 [==============================] - 0s 66us/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0107 - val_accuracy: 1.0000
Epoch 733/1000
331/331 [==============================] - 0s 71us/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0

Epoch 783/1000
331/331 [==============================] - 0s 73us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 784/1000
331/331 [==============================] - 0s 66us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 785/1000
331/331 [==============================] - 0s 79us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 786/1000
331/331 [==============================] - 0s 69us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 787/1000
331/331 [==============================] - 0s 75us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 788/1000
331/331 [==============================] - 0s 72us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 789/1000
331/331 [==============================] - 0s 68us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0

Epoch 839/1000
331/331 [==============================] - 0s 77us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 840/1000
331/331 [==============================] - 0s 69us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 841/1000
331/331 [==============================] - 0s 75us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 842/1000
331/331 [==============================] - 0s 70us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 843/1000
331/331 [==============================] - 0s 76us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 844/1000
331/331 [==============================] - 0s 73us/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 845/1000
331/331 [==============================] - 0s 69us/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0

Epoch 895/1000
331/331 [==============================] - 0s 73us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 896/1000
331/331 [==============================] - 0s 70us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 897/1000
331/331 [==============================] - 0s 71us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 898/1000
331/331 [==============================] - 0s 76us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0088 - val_accuracy: 1.0000
Epoch 899/1000
331/331 [==============================] - 0s 67us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 900/1000
331/331 [==============================] - 0s 66us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0088 - val_accuracy: 1.0000
Epoch 901/1000
331/331 [==============================] - 0s 70us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0

Epoch 951/1000
331/331 [==============================] - 0s 73us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 952/1000
331/331 [==============================] - 0s 69us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 953/1000
331/331 [==============================] - 0s 72us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 954/1000
331/331 [==============================] - 0s 70us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 955/1000
331/331 [==============================] - 0s 77us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 956/1000
331/331 [==============================] - 0s 68us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 957/1000
331/331 [==============================] - 0s 77us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0

In [32]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

143/143 [==============================] - 0s 56us/step
over-sampling test accuracy: 100.00%


In [51]:
#### #### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [52]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 664us/step - loss: 1.9874 - accuracy: 0.6556 - val_loss: 0.8193 - val_accuracy: 0.5035
Epoch 2/1000
331/331 [==============================] - 0s 127us/step - loss: 2.2351 - accuracy: 0.6677 - val_loss: 0.7616 - val_accuracy: 0.6224
Epoch 3/1000
331/331 [==============================] - 0s 131us/step - loss: 2.0700 - accuracy: 0.6616 - val_loss: 0.7180 - val_accuracy: 0.6923
Epoch 4/1000
331/331 [==============================] - 0s 132us/step - loss: 2.2126 - accuracy: 0.6949 - val_loss: 0.6923 - val_accuracy: 0.6643
Epoch 5/1000
331/331 [==============================] - 0s 146us/step - loss: 2.1212 - accuracy: 0.7341 - val_loss: 0.6635 - val_accuracy: 0.7203
Epoch 6/1000
331/331 [==============================] - 0s 153us/step - loss: 2.1593 - accuracy: 0.7251 - val_loss: 0.6433 - val_accuracy: 0.7552
Epoch 7/1000
331/331 [==============================] - 0s 199us/step - loss: 

331/331 [==============================] - 0s 99us/step - loss: 1.4878 - accuracy: 0.9003 - val_loss: 0.3830 - val_accuracy: 0.9441
Epoch 57/1000
331/331 [==============================] - 0s 89us/step - loss: 2.1290 - accuracy: 0.8580 - val_loss: 0.3793 - val_accuracy: 0.9441
Epoch 58/1000
331/331 [==============================] - 0s 94us/step - loss: 2.0411 - accuracy: 0.8640 - val_loss: 0.3771 - val_accuracy: 0.9441
Epoch 59/1000
331/331 [==============================] - 0s 97us/step - loss: 1.6191 - accuracy: 0.8973 - val_loss: 0.3734 - val_accuracy: 0.9371
Epoch 60/1000
331/331 [==============================] - 0s 95us/step - loss: 1.9461 - accuracy: 0.8792 - val_loss: 0.3730 - val_accuracy: 0.9441
Epoch 61/1000
331/331 [==============================] - 0s 93us/step - loss: 1.8918 - accuracy: 0.8761 - val_loss: 0.3671 - val_accuracy: 0.9371
Epoch 62/1000
331/331 [==============================] - 0s 88us/step - loss: 1.3825 - accuracy: 0.9124 - val_loss: 0.3678 - val_accuracy:

331/331 [==============================] - 0s 83us/step - loss: 1.4525 - accuracy: 0.9094 - val_loss: 0.2775 - val_accuracy: 1.0000
Epoch 113/1000
331/331 [==============================] - 0s 82us/step - loss: 1.8223 - accuracy: 0.8852 - val_loss: 0.2759 - val_accuracy: 1.0000
Epoch 114/1000
331/331 [==============================] - 0s 83us/step - loss: 1.8610 - accuracy: 0.8852 - val_loss: 0.2762 - val_accuracy: 1.0000
Epoch 115/1000
331/331 [==============================] - 0s 80us/step - loss: 1.8248 - accuracy: 0.8852 - val_loss: 0.2773 - val_accuracy: 1.0000
Epoch 116/1000
331/331 [==============================] - 0s 77us/step - loss: 2.1027 - accuracy: 0.8671 - val_loss: 0.2744 - val_accuracy: 1.0000
Epoch 117/1000
331/331 [==============================] - 0s 82us/step - loss: 1.5803 - accuracy: 0.9033 - val_loss: 0.2730 - val_accuracy: 1.0000
Epoch 118/1000
331/331 [==============================] - 0s 88us/step - loss: 1.5383 - accuracy: 0.9033 - val_loss: 0.2722 - val_acc

Epoch 168/1000
331/331 [==============================] - 0s 76us/step - loss: 1.8384 - accuracy: 0.8852 - val_loss: 0.2358 - val_accuracy: 1.0000
Epoch 169/1000
331/331 [==============================] - 0s 82us/step - loss: 1.8371 - accuracy: 0.8852 - val_loss: 0.2376 - val_accuracy: 1.0000
Epoch 170/1000
331/331 [==============================] - 0s 77us/step - loss: 1.9798 - accuracy: 0.8731 - val_loss: 0.2376 - val_accuracy: 1.0000
Epoch 171/1000
331/331 [==============================] - 0s 69us/step - loss: 2.0270 - accuracy: 0.8701 - val_loss: 0.2365 - val_accuracy: 1.0000
Epoch 172/1000
331/331 [==============================] - 0s 73us/step - loss: 1.8840 - accuracy: 0.8822 - val_loss: 0.2368 - val_accuracy: 1.0000
Epoch 173/1000
331/331 [==============================] - 0s 68us/step - loss: 1.9262 - accuracy: 0.8822 - val_loss: 0.2366 - val_accuracy: 1.0000
Epoch 174/1000
331/331 [==============================] - 0s 75us/step - loss: 1.6055 - accuracy: 0.8973 - val_loss: 0

Epoch 224/1000
331/331 [==============================] - 0s 74us/step - loss: 1.5444 - accuracy: 0.9033 - val_loss: 0.2212 - val_accuracy: 1.0000
Epoch 225/1000
331/331 [==============================] - 0s 81us/step - loss: 1.6858 - accuracy: 0.8943 - val_loss: 0.2215 - val_accuracy: 1.0000
Epoch 226/1000
331/331 [==============================] - 0s 77us/step - loss: 1.6388 - accuracy: 0.9003 - val_loss: 0.2199 - val_accuracy: 1.0000
Epoch 227/1000
331/331 [==============================] - 0s 70us/step - loss: 2.1507 - accuracy: 0.8640 - val_loss: 0.2209 - val_accuracy: 1.0000
Epoch 228/1000
331/331 [==============================] - 0s 76us/step - loss: 2.1053 - accuracy: 0.8671 - val_loss: 0.2208 - val_accuracy: 1.0000
Epoch 229/1000
331/331 [==============================] - 0s 68us/step - loss: 1.4532 - accuracy: 0.9094 - val_loss: 0.2206 - val_accuracy: 1.0000
Epoch 230/1000
331/331 [==============================] - 0s 74us/step - loss: 1.6359 - accuracy: 0.9003 - val_loss: 0

...

Epoch 728/1000
331/331 [==============================] - 0s 73us/step - loss: 1.4564 - accuracy: 0.9094 - val_loss: 0.1727 - val_accuracy: 1.0000
Epoch 729/1000
331/331 [==============================] - 0s 80us/step - loss: 1.5958 - accuracy: 0.9003 - val_loss: 0.1721 - val_accuracy: 1.0000
Epoch 730/1000
331/331 [==============================] - 0s 76us/step - loss: 1.6903 - accuracy: 0.8943 - val_loss: 0.1727 - val_accuracy: 1.0000
Epoch 731/1000
331/331 [==============================] - 0s 68us/step - loss: 1.6902 - accuracy: 0.8943 - val_loss: 0.1728 - val_accuracy: 1.0000
Epoch 732/1000
331/331 [==============================] - 0s 74us/step - loss: 1.9229 - accuracy: 0.8792 - val_loss: 0.1722 - val_accuracy: 1.0000
Epoch 733/1000
331/331 [==============================] - 0s 69us/step - loss: 1.6902 - accuracy: 0.8943 - val_loss: 0.1730 - val_accuracy: 1.0000
Epoch 734/1000
331/331 [==============================] - 0s 74us/step - loss: 1.4567 - accuracy: 0.9094 - val_loss: 0

Epoch 784/1000
331/331 [==============================] - 0s 72us/step - loss: 1.8750 - accuracy: 0.8822 - val_loss: 0.1712 - val_accuracy: 1.0000
Epoch 785/1000
331/331 [==============================] - 0s 81us/step - loss: 1.2216 - accuracy: 0.9245 - val_loss: 0.1699 - val_accuracy: 1.0000
Epoch 786/1000
331/331 [==============================] - 0s 77us/step - loss: 1.6880 - accuracy: 0.8943 - val_loss: 0.1720 - val_accuracy: 1.0000
Epoch 787/1000
331/331 [==============================] - 0s 70us/step - loss: 1.7819 - accuracy: 0.8882 - val_loss: 0.1714 - val_accuracy: 1.0000
Epoch 788/1000
331/331 [==============================] - 0s 75us/step - loss: 1.8281 - accuracy: 0.8852 - val_loss: 0.1709 - val_accuracy: 1.0000
Epoch 789/1000
331/331 [==============================] - 0s 68us/step - loss: 1.8743 - accuracy: 0.8822 - val_loss: 0.1710 - val_accuracy: 1.0000
Epoch 790/1000
331/331 [==============================] - 0s 74us/step - loss: 1.2220 - accuracy: 0.9245 - val_loss: 0

Epoch 840/1000
331/331 [==============================] - 0s 79us/step - loss: 1.3136 - accuracy: 0.9184 - val_loss: 0.1697 - val_accuracy: 1.0000
Epoch 841/1000
331/331 [==============================] - 0s 78us/step - loss: 1.5006 - accuracy: 0.9063 - val_loss: 0.1687 - val_accuracy: 1.0000
Epoch 842/1000
331/331 [==============================] - 0s 74us/step - loss: 1.2201 - accuracy: 0.9245 - val_loss: 0.1677 - val_accuracy: 1.0000
Epoch 843/1000
331/331 [==============================] - 0s 76us/step - loss: 1.6863 - accuracy: 0.8943 - val_loss: 0.1691 - val_accuracy: 1.0000
Epoch 844/1000
331/331 [==============================] - 0s 70us/step - loss: 1.2206 - accuracy: 0.9245 - val_loss: 0.1690 - val_accuracy: 1.0000
Epoch 845/1000
331/331 [==============================] - 0s 75us/step - loss: 1.5933 - accuracy: 0.9003 - val_loss: 0.1690 - val_accuracy: 1.0000
Epoch 846/1000
331/331 [==============================] - 0s 73us/step - loss: 1.8724 - accuracy: 0.8822 - val_loss: 0

Epoch 896/1000
331/331 [==============================] - 0s 77us/step - loss: 1.4980 - accuracy: 0.9063 - val_loss: 0.1675 - val_accuracy: 1.0000
Epoch 897/1000
331/331 [==============================] - 0s 72us/step - loss: 2.0109 - accuracy: 0.8731 - val_loss: 0.1666 - val_accuracy: 1.0000
Epoch 898/1000
331/331 [==============================] - 0s 75us/step - loss: 1.8247 - accuracy: 0.8852 - val_loss: 0.1668 - val_accuracy: 1.0000
Epoch 899/1000
331/331 [==============================] - 0s 75us/step - loss: 1.4055 - accuracy: 0.9124 - val_loss: 0.1675 - val_accuracy: 1.0000
Epoch 900/1000
331/331 [==============================] - 0s 79us/step - loss: 1.5444 - accuracy: 0.9033 - val_loss: 0.1678 - val_accuracy: 1.0000
Epoch 901/1000
331/331 [==============================] - 0s 77us/step - loss: 1.6852 - accuracy: 0.8943 - val_loss: 0.1674 - val_accuracy: 1.0000
Epoch 902/1000
331/331 [==============================] - 0s 72us/step - loss: 1.7777 - accuracy: 0.8882 - val_loss: 0

Epoch 952/1000
331/331 [==============================] - 0s 72us/step - loss: 1.7768 - accuracy: 0.8882 - val_loss: 0.1651 - val_accuracy: 1.0000
Epoch 953/1000
331/331 [==============================] - 0s 81us/step - loss: 1.8232 - accuracy: 0.8852 - val_loss: 0.1663 - val_accuracy: 1.0000
Epoch 954/1000
331/331 [==============================] - 0s 78us/step - loss: 1.4965 - accuracy: 0.9063 - val_loss: 0.1667 - val_accuracy: 1.0000
Epoch 955/1000
331/331 [==============================] - 0s 70us/step - loss: 1.4964 - accuracy: 0.9063 - val_loss: 0.1674 - val_accuracy: 1.0000
Epoch 956/1000
331/331 [==============================] - 0s 75us/step - loss: 1.4966 - accuracy: 0.9063 - val_loss: 0.1665 - val_accuracy: 1.0000
Epoch 957/1000
331/331 [==============================] - 0s 69us/step - loss: 1.3572 - accuracy: 0.9154 - val_loss: 0.1661 - val_accuracy: 1.0000
Epoch 958/1000
331/331 [==============================] - 0s 74us/step - loss: 2.1495 - accuracy: 0.8640 - val_loss: 0

In [54]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 84us/step
over-sampling test accuracy: 100.00%


In [37]:
############ Logistic Regression ############

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [39]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 97.71%


In [41]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 100.00%


In [43]:
############## Random Forest ##############

In [44]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [45]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [46]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [47]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [48]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [49]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.   1.   1.   1.   0.95]
0.99


In [50]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[1.         1.         0.98484848 0.98484848 0.98484848]
0.990909090909091
